# cherrypy

upload file in cherrypy

In [1]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer
import gensim
from gensim.models import Word2Vec 

ps=PorterStemmer()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rameshwaram\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def readf(filename):
    file=open(filename,encoding='utf-8')
    read=file.read()
    file.seek(0)
   
   

    # create array
    array=sent_tokenize(read)
    line= len(array)
    
    # remove punctuation
    punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~'''
    for ele in read:  
        if ele in punc:  
            read = read.replace(ele, " ")  
 
    read=read.lower()
    
    # remove stopwords
    text_tokens = word_tokenize(read)
    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
    
    # remove digits
    tokens=[]
    for i in range(len(tokens_without_sw)):
                   if tokens_without_sw[i].isdigit()==False:
                       tokens.append(tokens_without_sw[i])
                    
    # stemming
    ps=PorterStemmer()
    for i in range(len(tokens)):
        tokens[i]=ps.stem(tokens[i])
        
    #set
    newset=set(tokens)
    
    #dictionary
    dt={}

    for i in range(line):
        check=array[i].lower()
        for item in newset:

            if item in check:
                if item not in dt:
                    dt[item]=[]

                if item in dt:
                    dt[item].append(i+1)

    return dt



In [3]:
def lines(b,a):
    d=[]
    for i in range(len(b)):
        d.append("Line number {} : {} ".format(b[i],a[b[i]-1]))
    return(d)

    
def readfil(filename):
    file=open(filename,encoding='utf-8')
    read=file.read()
    file.seek(0)
   
   

    # create array
    array=sent_tokenize(read)
    
    return array
def setor(a,b):
    myset1=set(a)
    myset2=set(b)
    myset3= myset1|myset2
    return myset3

def setand(a,b):
    myset1=set(a)
    myset2=set(b)
    myset3= myset1&myset2
    return myset3

def precedence(op): 
      
    if op == '&' or op == '|': 
        return 2
    if op == '!': 
        return 1
    return 0

def applyOp(a,b,op):
    if op=='&':
        return setand(a,b)
    if op=='|':
        return setor(a,b)
    if op=='!':
        return setnot(a,b)
    
def setnot(a,b):
    myset1=set(a)
    myset2=set(b)
    myset3= myset1-myset2
    return myset3
    
    
operator=['|', '&','(',')','!']
def evaluate(tokens,filename,dt): 
    
    dt=readf(filename)
      
    values = []     
    ops = [] 
    i = 0
      
    while i < len(tokens):           
       
        if tokens[i] == ' ': 
            i += 1
            continue          
      
        elif tokens[i] == '(': 
            ops.append(tokens[i]) 
            
        elif tokens[i] not in operator : 
                      
            values.append(dt[tokens[i]]) 
        elif tokens[i] == ')': 
          
            while len(ops) != 0 and ops[-1] != '(': 
              
                val2 = values.pop() 
                val1 = values.pop() 
                op = ops.pop() 
                  
                values.append(applyOp(val1, val2, op)) 
              
            
            ops.pop() 
            
            
        else: 
          
           
            while (len(ops) != 0 and
                precedence(ops[-1]) >= precedence(tokens[i])): 
                          
                val2 = values.pop() 
                val1 = values.pop() 
                op = ops.pop() 
                  
                values.append(applyOp(val1, val2, op)) 
              
           
            ops.append(tokens[i]) 
          
        i += 1
    while len(ops) != 0: 
          
        val2 = values.pop() 
        val1 = values.pop() 
        op = ops.pop() 
                  
        values.append(applyOp(val1, val2, op)) 
      
    
    return values[0]     


## For similar word

In [4]:
# def applywv(filename,word):
#     try:
#         f=open(filename, encoding='utf-8')
#         read=sent_tokenize(f.read())
#         sentence=[word_tokenize(i) for i in read]
#         punc = '''!()“-[]{};:'"\, <>./?@#$%^&*_~'''
#         for i in range(len(sentence)):
#             sentence[i]=[word.lower() for word in sentence[i] if word.lower() not in stopwords.words()]

#         for i in range(len(sentence)):
#             sentence[i]=[word for word in sentence[i] if word not in punc]
#             sentence[i]=[word for word in sentence[i] if word.isalpha()]



#         vector=Word2Vec(sentence, min_count=1,size=2)
#         similar= vector.wv.most_similar(word)
#         arr=[]

#         for i in range(10):
#             arr.append(similar[i][0])

#         return arr
#     except:
#         return "invalid"



In [5]:
def linescontent(b,a):
    d=[]
    for i in range(len(b)):
        d.append(a[b[i]-1])
    return(d)

In [6]:
def maximum_list(l):
    maximum = max(l)
    array=[i+1 for i in range(len(l)) if l[i]==maximum ]
    
    return array
    

## For similar sentence

In [7]:
def clustering(filename,comparison):
    f=open(filename)
    a=sent_tokenize(f.read())
    stop_words = list(stopwords.words('english'))
    punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~'''
    s=[(word_tokenize(a[i])) for i in range(len(a))]
    outer_1=[]

    for i in range(len(s)):
        inner_1=[]
        for j in range(len(s[i])):
            if s[i][j] not in (punc or stop_words):
                s[i][j]=ps.stem(s[i][j])
                if s[i][j] not in stop_words:
                    inner_1.append(s[i][j].lower())
        outer_1.append(set(inner_1))
    rvector=outer_1[0]
    for i in range(1,len(s)):
        rvector=rvector.union(outer_1[i])
    outer=[]
    for i in range(len(outer_1)):
        inner=[]
        for w in rvector:        
            if w in outer_1[i]:
                inner.append(1)           
            else:
                inner.append(0)
        outer.append(inner)
   
    
    check=(word_tokenize(comparison))
    check=[ps.stem(check[i]).lower() for i in range(len(check))]

    

    check1=[]
    for w in rvector: 
        if w in check: check1.append(1) # create a vector 
        else:check1.append(0)
    
    ds=[]        
            
    for j in range(len(outer)):
        similarity_index=0
        c=0
        if check1==outer[j]:
            ds.append(0)
        else:
            for i in range(len(rvector)): 
             
                c+= check1[i]*outer[j][i] 
       
       
            similarity_index+=c
            ds.append(similarity_index)
        
        
    result=maximum_list(ds)
    return result
 

In [8]:
import random
import string
import cherrypy
import nltk
import time
from cherrypy import tools


class Root(object):
   
    @cherrypy.expose
    def index(self):
        return """<html>
<head>
  <style>
    .container{
      height: 300px;
      width: 600px;
      background-color: maroon;
      margin-top: 12%;
      margin-left: 25%;
      color: white;
    }
    .container h2{
      font-size: 40px;
      text-align: center;  
      
    }
    .off{
      font-size: 25px;
      margin-left: 23%;
      color: yellow;
      background-color: ;
    }
    .on{
      font-size: 25px;
      border-top-right-radius: 25px;
      border-top-left-radius: 25px;
      border-bottom-left-radius: 25px;
      border-bottom-right-radius: 25px;
      margin-top: 20px;   }
  </style>
  </head>
<body>

  <div class="container">  
    <h2><u><i>Upload a file</i></u></h2>
    <form action="store" id="form" method="GET">
    <input class="off" type="file" name="myFile" /><br />
    <input style="margin-left: 50px; height: 40px; width: 500px;" class="on" type="text" name="search" placeholder="Enter the query............................................."><br>
    <input style="margin-left: 250px;" class="on" id=" submit" type="submit"/></div>
  </div>    
    </form>
  </div>
            
</body>
</html>"""
   
    
    @cherrypy.expose
    def store(self, myFile,search):
        
        b=str(readf(myFile))
        c=[]
        tokens=word_tokenize(search)
        for i in range(len(tokens)):
            tokens[i]=ps.stem(tokens[i])
        numb1=evaluate(tokens,myFile,c)
        change=list(numb1)
        filelist= readfil(myFile)
        d= lines(change,filelist)
        similar=[]
        
        for i in range(len(d)):
            similar.append(clustering(myFile,d[i]))
        obc=[]
        for j in range(len(similar)):
            obc.append(linescontent(similar[j],filelist))
            
        
        
        
        
        out= """<html>
        <body>
        <p style="color:brown; font-size: 30px;">Dictionary</u></p><br>
        <p> %s</p><br><br>
        <p style="color:brown; font-size: 30px;"><u> Searching for :  </u> %s</p><br><br>
       
        <p style="color:brown; font-size: 30px;"><u>Found in these lines:-</u></p><br>
        <p>%s</p><br><br>
        <p> Output of query: %s </p>
        <p> Similar: %s</p>
       
        
        <a style="color:red; font-size:35px;" id="shutdown"; href="./shutdown"><i>Shutdown Server</i></a>
        </body>
        </html>
        
        """
       
        return out % (b,search,str(d),str(numb1),str(obc))
  
    
    @cherrypy.expose
    def shutdown(self):
        cherrypy.engine.exit()

if __name__=="__main__":
    cherrypy.config.update({'server.socket_port': 8087})
    
    cherrypy.quickstart(Root())


[25/Jul/2020:17:55:14] ENGINE Listening for SIGTERM.
[25/Jul/2020:17:55:14] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[25/Jul/2020:17:55:14] ENGINE Set handler for console events.
[25/Jul/2020:17:55:14] ENGINE Started monitor thread 'Autoreloader'.
[25/Jul/2020:17:55:15] ENGINE Serving on http://127.0.0.1:8087
[25/Jul/2020:17:55:15] ENGINE Bus STARTED


127.0.0.1 - - [25/Jul/2020:17:55:17] "GET / HTTP/1.1" 200 1126 "http://localhost:8888/notebooks/cherrypy.ipynb" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"
127.0.0.1 - - [25/Jul/2020:17:55:39] "GET /store?myFile=new.txt&search=%28+place+%7C+goa+%29 HTTP/1.1" 200 935 "http://127.0.0.1:8087/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"


[25/Jul/2020:17:56:16] ENGINE Bus STOPPING
[25/Jul/2020:17:56:16] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8087)) shut down
[25/Jul/2020:17:56:16] ENGINE Stopped thread 'Autoreloader'.
[25/Jul/2020:17:56:16] ENGINE Removed handler for console events.
[25/Jul/2020:17:56:16] ENGINE Bus STOPPED
[25/Jul/2020:17:56:16] ENGINE Bus EXITING
[25/Jul/2020:17:56:16] ENGINE Waiting for child threads to terminate...
[25/Jul/2020:17:56:16] ENGINE Bus EXITED


127.0.0.1 - - [25/Jul/2020:17:56:16] "GET /shutdown HTTP/1.1" 200 - "http://127.0.0.1:8087/store?myFile=new.txt&search=%28+place+%7C+goa+%29" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"


[25/Jul/2020:17:56:16] ENGINE Waiting for thread CP Server Thread-13.
